**6. Computing VAEP from Wyscout data and comabining it with our metrics dataset (as generated from '3_metricscollection' notebook)**

The following tasks were taken into account in this notebook:

1. To convert the Wyscout data to SPADL format and to compute VAEP values using the Socceractions framework

2. Combine the VAEP values (as well as offensive and defensive values) with the existing metrics dataframe

The following results were saved as pickle files:

1. Cluster wise dataframes with VAEP, offensive and defensive value as additional features


# Imports 

In [2]:
!pip install tables==3.6.1
!pip install socceraction==0.2.0

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import seaborn as sns
from math import *
import matplotlib.pylab as pyl
import pickle
import swifter
import warnings
import plotly.express as px
from itertools import chain
import scipy.stats as sps
from tqdm import tqdm
from unidecode import unidecode
import re
from io import BytesIO
from pathlib import Path
from tqdm.notebook import tqdm
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile
import pandas as pd
from sklearn.metrics import brier_score_loss, roc_auc_score  # version 0.22.2
from xgboost import XGBClassifier  # version 1.0.2

import socceraction.vaep.features as features
import socceraction.vaep.labels as labels

from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value

C:\Users\Anuj\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#pd.set_option('max_colwidth', 999)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)

In [3]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

# Preprocess the data

## Preprocess the Wyscout data

In [6]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

### Teams

In [8]:
json_teams = read_json_file('../../data/teams/teams.json')
df_teams = pd.read_json(json_teams)

In [9]:
df_teams.head(10)

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alavés,696,Deportivo Alavés,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
5,Troyes,Troyes,3795,Espérance Sportive Troyes Aube Champagne,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
6,Getafe (Madrid),Getafe,698,Getafe Club de Fútbol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
7,Mönchengladbach,Borussia M'gladbach,2454,Borussia VfL Mönchengladbach,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club
8,"Huddersfield, West Yorkshire",Huddersfield Town,1673,Huddersfield Town FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
9,Bilbao,Athletic Club,678,Athletic Club Bilbao,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [10]:
df_teams.to_hdf('wyscout.h5', key='teams', mode='w')

### Players

In [11]:
json_players = read_json_file('../../data/players/players.json')
df_players = pd.read_json(json_players)

In [12]:
df_players.head(10)

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konaté,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konaté,null
5,"{'name': 'Netherlands', 'id': '528', 'alpha3co...",83,Jasper,,Cillessen,676,1989-04-22,185,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Netherlands', 'id': '528', 'alpha3co...",33,right,J. Cillessen,664
6,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",91,Toby,,Alderweireld,1624,1989-03-02,187,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",36,right,T. Alderweireld,5629
7,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",88,Jan,,Vertonghen,1624,1987-04-24,189,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",48,left,J. Vertonghen,5629
8,"{'name': 'France', 'id': '250', 'alpha3code': ...",74,Alexander,,Djiku,3783,1994-08-09,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",229427,right,A. Djiku,null
9,"{'name': 'Denmark', 'id': '208', 'alpha3code':...",76,Christian,,Dannemann Eriksen,1624,1992-02-14,180,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Denmark', 'id': '208', 'alpha3code':...",54,right,C. Eriksen,7712


In [13]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')

### Matches

In [15]:
competitions = [
     'England',
     'France',
     'Germany',
     'Italy',
     'Spain',
    'European Championship',
     'World Cup'
]

In [16]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'../../data/matches/matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [17]:
df_matches.head(10)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId,groupName
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364,NaN
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364,NaN
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364,NaN
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364,NaN
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364,NaN
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"Newcastle United - Chelsea, 3 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364,NaN
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"Southampton - Manchester City, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364,NaN
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"Swansea City - Stoke City, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364,NaN
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"Tottenham Hotspur - Leicester City, 5 - 4","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364,NaN
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"West Ham United - Everton, 3 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364,NaN


In [18]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

### Events

In [19]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'../../data/events/events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

## Convert the Wyscout data to the SPADL representation

In [20]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games


  0%|          | 0/1941 [00:00<?, ?game/s]

...Converting players
...Converting teams
...Generating player_games


  0%|          | 0/1941 [00:00<?, ?game/s]

...Converting events to actions


100%|██████████| 1941/1941 [1:11:39<00:00,  2.21s/game]


# Value game states

In [21]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [22]:
nb_prev_actions = 3

## Generate game state features

In [23]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

In [24]:
df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{2500089}')

In [25]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

## Generate game state labels

In [26]:
functions_labels = [
    labels.scores,
    labels.concedes
]

In [27]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

## Generate dataset

In [28]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)

In [29]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

In [30]:
df_features.head(10)

,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,type_interception_a0,type_shot_a0,type_shot_penalty_a0,type_shot_freekick_a0,type_keeper_save_a0,type_keeper_claim_a0,type_keeper_punch_a0,type_keeper_pick_up_a0,type_clearance_a0,type_bad_touch_a0,type_non_action_a0,type_dribble_a0,type_goalkick_a0,type_pass_a1,type_cross_a1,type_throw_in_a1,type_freekick_crossed_a1,type_freekick_short_a1,type_corner_crossed_a1,type_corner_short_a1,type_take_on_a1,type_foul_a1,type_tackle_a1,type_interception_a1,type_shot_a1,type_shot_penalty_a1,type_shot_freekick_a1,type_keeper_save_a1,type_keeper_claim_a1,type_keeper_punch_a1,type_keeper_pick_up_a1,type_clearance_a1,type_bad_touch_a1,type_non_action_a1,type_dribble_a1,type_goalkick_a1,type_pass_a2,type_cross_a2,type_throw_in_a2,type_freekick_crossed_a2,type_freekick_short_a2,type_corner_crossed_a2,type_corner_short_a2,type_take_on_a2,type_foul_a2,type_tackle_a2,type_interception_a2,type_shot_a2,type_shot_penalty_a2,type_shot_freekick_a2,type_keeper_save_a2,type_keeper_claim_a2,type_keeper_punch_a2,type_keeper_pick_up_a2,type_clearance_a2,type_bad_touch_a2,type_non_action_a2,type_dribble_a2,type_goalkick_a2,bodypart_foot_a0,bodypart_head_a0,bodypart_other_a0,bodypart_foot_a1,bodypart_head_a1,bodypart_other_a1,bodypart_foot_a2,bodypart_head_a2,bodypart_other_a2,result_fail_a0,result_success_a0,result_offside_a0,result_owngoal_a0,result_yellow_card_a0,result_red_card_a0,result_fail_a1,result_success_a1,result_offside_a1,result_owngoal_a1,result_yellow_card_a1,result_red_card_a1,result_fail_a2,result_success_a2,result_offside_a2,result_owngoal_a2,result_yellow_card_a2,result_red_card_a2,goalscore_team,goalscore_opponent,goalscore_diff,start_x_a0,start_y_a0,start_x_a1,start_y_a1,start_x_a2,start_y_a2,end_x_a0,end_y_a0,end_x_a1,end_y_a1,end_x_a2,end_y_a2,dx_a0,dy_a0,movement_a0,dx_a1,dy_a1,movement_a1,dx_a2,dy_a2,movement_a2,dx_a01,dy_a01,mov_a01,dx_a02,dy_a02,mov_a02,start_dist_to_goal_a0,start_angle_to_goal_a0,start_dist_to_goal_a1,start_angle_to_goal_a1,start_dist_to_goal_a2,start_angle_to_goal_a2,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,0,0,0,52.50,34.00,52.50,34.00,52.50,34.00,42.00,37.40,42.00,37.40,42.00,37.40,-10.50,3.40,11.036757,-10.50,3.40,11.036757,-10.50,3.40,11.036757,-1.050000e+01,3.400000e+00,1.103676e+01,-10.50,3.400000e+00,11.036757,52.500000,0.000000,52.500000,0.000000,52.500000,0.000000,63.091679,0.053916,63.091679,0.053916,63.091679,0.053916,True,True,0.000000,0.000000
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,0,0,0,42.00,37.40,52.50,34.00,52.50,34.00,40.95,57.80,42.00,37.40,42.00,37.40,-1.05,20.40,20.427004,-10.50,3.40,11.036757,-10.50,3.40,11.036757,0.

In [31]:
columns_labels = [
    'scores',
    'concedes'
]

In [32]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

In [33]:
df_labels.tail(10)

,scores,concedes
2465146,True,False
2465147,True,False
2465148,True,False
2465149,True,False
2465150,True,False
2465151,True,False
2465152,False,True
2465153,False,True
2465154,True,False
2465155,False,False


## Train classifiers

In [34]:
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(n_estimators=100, max_depth=4)
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

In [35]:
filename = '../../data/finalised_vaep_model.pkl'
pickle.dump(model,open(filename,'wb'))

## Estimate probabilities

In [36]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [37]:
df_predictions.head(10)

,scores,concedes
0,0.003628,0.001496
1,0.005049,0.001750
2,0.003883,0.002318
3,0.014610,0.002321
4,0.018392,0.001865
5,0.020405,0.001477
6,0.011381,0.002902
7,0.001521,0.022920
8,0.021142,0.002600
9,0.001294,0.008486


The following cell obtains the `game_id` for each action in order to store the predictions per game.

In [38]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

In [39]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)

In [40]:
df_predictions.head(10)

,scores,concedes,game_id
0,0.003628,0.001496,2500089
1,0.005049,0.001750,2500089
2,0.003883,0.002318,2500089
3,0.014610,0.002321,2500089
4,0.018392,0.001865,2500089
5,0.020405,0.001477,2500089
6,0.011381,0.002902,2500089
7,0.001521,0.022920,2500089
8,0.021142,0.002600,2500089
9,0.001294,0.008486,2500089


In [41]:
df_predictions_per_game = df_predictions.groupby('game_id')

In [42]:
for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

# Value on-the-ball actions - VAEP Values

<img src="./expl_photo/1.JPG">
<img src="./expl_photo/2.JPG">
<img src="./expl_photo/3.JPG">
<img src="./expl_photo/4.JPG">

In [43]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [44]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

In [45]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [46]:
pl_teams = ['Arsenal','Leicester City','Manchester City','Brighton & Hove Albion','Burnley','Chelsea',
            'Crystal Palace','Huddersfield Town','Everton','Stoke City','Manchester United','West Ham United',
            'Newcastle United','Tottenham Hotspur','Swansea City','Southampton','Watford','Liverpool',
            'West Bromwich Albion','AFC Bournemouth']

In [47]:
df_values_pl = df_values.loc[df_values['short_team_name'].isin(pl_teams)]

In [49]:
# df_values_pl.loc[(df_values_pl['game_id']==2499719.0)]

* scores = P_scores(Si, x) - prob(scoring in the next 10 actions) - obtained from the ML Model - given for the particular `state`
* concedes = P_concedes(Si, x) - prob(concedes in the next 10 actions) - obtained from the ML Model - given for the particular `state`
* offensive_value = P_scores(Si) - P_scores(Si-1) - given for the particular `action`
* defensive_value = P_concedes(Si-1) - P_concedes(Si) - given for the particular `action`

**Creating DF for VAEP and Offensive value sums per team per match**

In [24]:
from sklearn.preprocessing import MinMaxScaler
df_values_pl['offensive_value_norm'] = MinMaxScaler().fit_transform(df_values_pl['offensive_value'].values.reshape(-1,1))
df_values_pl['vaep_value_norm'] = MinMaxScaler().fit_transform(df_values_pl['vaep_value'].values.reshape(-1,1))

In [25]:
df_values_pl.head()

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,type_name,result_name,bodypart_name,short_name,first_name,last_name,birth_date,short_team_name,team_name,scores,concedes,offensive_value,defensive_value,vaep_value,offensive_value_norm,vaep_value_norm
143676,2499719.0,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,0,1,pass,success,foot,A. Lacazette,Alexandre,Lacazette,1991-05-28,Arsenal,Arsenal FC,0.003792,0.000771,0.000000,-0.000000,0.000000,0.437685,0.442156
143677,2499719.0,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,0,1,pass,success,foot,R. Holding,Rob,Holding,1995-09-20,Arsenal,Arsenal FC,0.006808,0.001816,0.003016,-0.001046,0.001971,0.439391,0.443263
143678,2499719.0,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,0,1,pass,success,head,M. Özil,Mesut,Özil,1988-10-15,Arsenal,Arsenal FC,0.005715,0.001805,-0.001093,0.000011,-0.001082,0.437067,0.441549
143679,2499719.0,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,0,1,pass,success,head,Mohamed Elneny,Mohamed Naser,Elsayed Elneny,1992-07-11,Arsenal,Arsenal FC,0.005394,0.001754,-0.000321,0.000051,-0.000270,0.437504,0.442005
143680,2499719.0,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,0,1,pass,success,foot,Bellerín,Héctor,Bellerín Moruno,1995-03-19,Arsenal,Arsenal FC,0.013657,0.001785,0.008263,-0.000031,0.008232,0.442357,0.446780


In [27]:
df_values_pl.to_pickle('../../data/vaep/vaep_values.pkl')

# VAEP for Defenders 

## Pre-Processing 

In [61]:
vaep_values = pd.read_pickle('../../data/vaep/vaep_values.pkl')
match_def = pd.read_pickle("../../data/matches/match+def_lineup+footedness_ver2.pkl")

In [62]:
vaep_values = vaep_values.replace({'short_team_name':{
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }}
)

vaep_values['game_id'] = vaep_values['game_id'].astype(int)
vaep_values['temp'] = vaep_values['game_id'].astype(str) + vaep_values['short_team_name']
match_def['temp'] = match_def['wyId'].astype(str) + match_def['team']

In [63]:
vaep_values = vaep_values.merge(match_def[['temp', 'footedness']], left_on='temp', right_on='temp', how='left')
vaep_values.drop(columns = ['temp'], inplace=True)

In [64]:
vaep_values['name'] = vaep_values['first_name']+vaep_values['last_name']
vaep_values['name'] = vaep_values['name'].astype(str).apply(lambda x: unidecode(x))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace('-', ''))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace(' ', ''))
# vaep_values.loc[(vaep_values['last_name'].str.contains('PhilJa'))]

In [65]:
# Creating a key to identify each row using game_id, time_seconds and player name
vaep_values['key'] = vaep_values['game_id'].astype(str) + np.round(vaep_values['time_seconds'],6).astype(str) +vaep_values['name'].astype(str)

In [66]:
# vaep_values.loc[(vaep_values['name'].str.contains('AaronCre'))& (vaep_values['type_name']=='pass') &(vaep_values['game_id']==2499724)]


In [67]:
# Filtering the events that are labelled as pass and cross
vaep_values_pass = vaep_values.loc[(vaep_values['type_name'] == 'pass') | (vaep_values['type_name'] =='cross') ]

In [68]:
# Merging player roles to SPADL events data
df_players = pd.read_pickle('../../data/players/players.pkl')
roles_temp = df_players['role'].values
roles = list()
for i in roles_temp:
    roles.append(i['code3'])
players_roles = list(zip(roles,df_players['wyId'],df_players['playerName']))
df_players_roles = pd.DataFrame(players_roles,columns = ['role','playerId','playerName1'])
vaep_values_pass_proles = vaep_values_pass.merge(df_players_roles, left_on = 'player_id', right_on = 'playerId')
vaep_values_pass_proles.drop(['playerName1'], axis = 1, inplace = True)
vaep_values_pass_def = vaep_values_pass_proles.loc[vaep_values_pass_proles['role']=='DEF']

In [69]:
# Counting duplicate keys in the VAEP filtered dataframe. 
# Note: Some events in the dataframe have same key attributes (game_id, time_seconds and player name)
# When checked at a deeper level, these passes (roughly 30 of them) have the same player performing two 
# different passes from the same starting location to different end locations at the same time stamp.
# We have ignored these 30 passes from our analysis
vaep_values_pass_def['key'].value_counts()

2499949222.306174LaurentKoscielny        2
2499949467.025053ShkodranMustafi         2
2499949252.025934LaurentKoscielny        2
2499949433.742393LaurentKoscielny        2
2499949234.956542HectorBellerinMoruno    2
                                        ..
2500039898.001675SteveCook               1
24998452206.614498WesleyHoedt            1
2499898504.905296ChrisSmalling           1
24997551091.908577JackStephens           1
25000072551.497334SergeAurier            1
Name: key, Length: 141940, dtype: int64

In [70]:
# vaep_values_pass_def.loc[vaep_values_pass_def['key'].str.contains('2499949256.961055LaurentKoscielny')]

In [71]:
# Importing Wyscout events data
df_events_wyscout = pd.read_pickle('../../data/events/events_v2.pkl')

In [72]:
# Filtering out passes events which were performed by defenders in Wyscout events data
df_events_wyscout_pass = df_events_wyscout.loc[(df_events_wyscout['eventName']=='Pass') & (df_events_wyscout['role']=='DEF')]

In [73]:
# Creating a key to identify each row using matchid, event_seconds and player name
df_events_wyscout_pass['key'] = df_events_wyscout_pass['matchId'].astype(str) + np.round(df_events_wyscout_pass['eventSec'],6).astype(str) + df_events_wyscout_pass['playerName'].astype(str)

C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
# df_events_wyscout_pass.loc[df_events_wyscout_pass['key'].str.contains('2499949256.961055LaurentKoscielny')]

In [74]:
# Appending VAEP values along with offensive and defensive values to Wyscout events data from VAEP data
# by joining the two dataframes
df_events_vaep = df_events_wyscout_pass.merge(vaep_values_pass_def[[
    'key', 'vaep_value', 'offensive_value', 'defensive_value',
    'vaep_value_norm', 'offensive_value_norm'
]],
                                              how='left',
                                              left_on='key',
                                              right_on='key')

In [68]:
# Total number of passes in Wyscout events data['']
# df_events_vaep.loc[(df_events_vaep['playerName'].str.contains("AaronCres")) & ((df_events_vaep['eventName'].str.contains("Pass"))) & (df_events_vaep['matchId']==2500098)]

## Metrics Collection & Clustering

In [76]:
footedness_patterns = match_def["footedness"].unique()

In [77]:
match_def.rename(columns={'R-CB':'R_CB',"L-CB":'L_CB'},inplace=True)

**Creating seperate dataframes for each defensive lineup based for preferred foot of each defender**

In [78]:
df_rrrl = match_def.loc[match_def['footedness']=='right-right-right-left']
df_rrll = match_def.loc[match_def['footedness']=='right-right-left-left']
df_rrl = match_def.loc[match_def['footedness']=='right-right-left']
df_rrr = match_def.loc[match_def['footedness']=='right-right-right']
df_rll = match_def.loc[match_def['footedness']=='right-left-left']
df_rrrll = match_def.loc[match_def['footedness']=='right-right-right-left-left']
df_rrlr = match_def.loc[match_def['footedness']=='right-right-left-right']
df_rrrr = match_def.loc[match_def['footedness']=='right-right-right-right']
df_rrrrl = match_def.loc[match_def['footedness']=='right-right-right-right-left']
df_rlr = match_def.loc[match_def['footedness']=='right-left-right']
df_rrrlr = match_def.loc[match_def['footedness']=='right-right-right-left-right']
df_rrlll = match_def.loc[match_def['footedness']=='right-right-left-left-left']
df_rlll = match_def.loc[match_def['footedness']=='right-left-left-left']

**Creating a list of such dataframes**

In [79]:
df_clusters = [df_rrrl,df_rrll,df_rrl,df_rrr,df_rll,df_rrrll,df_rrlr,df_rrrr,df_rrrrl,df_rlr,df_rrrlr,df_rrlll,df_rlll]

**Creating a dictionary of mapping of players with a mismatch in names in events data and Premier League parsed data**

In [80]:
player_map = {  'RamiroFunesMori': 'JoseRamiroFunesMori',
                'KurtZouma': 'KurtHappyZouma',
                'Danilo': 'DaniloLuizdaSilva',
                'CesarAzpilicueta': 'CesarAzpilicuetaTanco',
                'EzequielSchelotto': 'MatiasEzequielSchelotto',
                'GaetanBong': 'GaetanBongSongo',
                'HectorBellerin': 'HectorBellerinMoruno',
                'AhmedHegazi': 'AhmedHegazy',
                'JamaalLascelles': 'JamalLascelles',
                'AngelRangel': 'AngelRangelZaragoza',
                'Zanka': 'MathiasJattahNjieJorgensen',
                'ChrisLwe': 'ChrisLowe',
                'EricBailly': 'EricBertrandBailly',
                'MarcosRojo': 'FaustinoMarcosAlbertoRojo',
                'CdricSoares': 'CedricRicardoAlvesSoares',
                'AngeloOgbonna': 'AngeloObinzeOgbonna',
                'HctorBellern': 'HectorBellerinMoruno',
                'DavinsonSanchez': 'DavinsonSanchezMina',
                'JavierManquillo': 'JavierManquilloGaitan',
                'TommySmith': 'TomSmith',
                'Bruno': 'BrunoSaltorGrau',
                'GatanBong': 'GaetanBongSongo',
                'NicolsOtamendi': 'NicolasHernanOtamendi',
                'CsarAzpilicueta': 'CesarAzpilicuetaTanco',
                'AntonioRdiger': 'AntonioRudiger',
                'JosHolebas': 'JoseHolebas',
                'SamusColeman': 'SeamusColeman',
                'AllanRomoNyom': 'AllanRomeoNyom',
                'NathanAk': 'NathanAke',
                'JosephGomez': 'JoeGomez',
                'AlbertoMoreno':'AlbertoMorenoPerez',
                'LuisAntonioValencia':'LuisAntonioValenciaMosquera',
                'VictorLindelf':'VictorNilssonLindelof',
                'DavinsonSnchez':'DavinsonSanchezMina',
                'NicolasOtamendi':'NicolasHernanOtamendi',
                'NachoMonreal':'IgnacioMonrealEraso',
                'FedericoFernndez':'FedericoFernandez',
                'SebastianPrdl':'SebastianProdl',
                'CedricSoares':'CedricRicardoAlvesSoares',
                'JoelMatip':'JoelAndreJobMatip',
                'MiguelBritos':'MiguelAngelBritosCabrera',
                'VictorLindelof':'VictorNilssonLindelof',
                'JamesCollins':'JamesMichaelCollins',
                'CucoMartina':'RhuendlyMartina',
                'DavidLuiz':'DavidLuizMoreiraMarinho',
                'MollaWagu':'MollaWague',
                'JrmyPied':'JeremyPied',
                'ChancelMbemba':'ChancelMbembaMangulu',
                'PabloZabaleta':'PabloJavierZabaletaGirod',
                'KikoFemenia':'FranciscoFemeniaFar',
                'CheikhouKouyat':'CheikhouKouyate',
                'KikoFemena':'FranciscoFemeniaFar',
                'JoseFonte':'JoseMigueldaRochaFonte',
                'JosFonte':'JoseMigueldaRochaFonte',
                'JesusGamez':'JesusGamezDuarte'}


**Creating a metrics collection function that takes in x (match_id) and y (player name) and returns the standard metrics-**

In [81]:
def getmetrics(x, y):
    try:
        y = player_map[y]
    except:
        pass
    split_y = re.findall('[A-Z][^A-Z]*', y)
    try:
        pass_df = df_events_vaep.loc[
            (df_events_vaep['playerName'].str.contains(split_y[-1]))
            & (df_events_vaep['playerName'].str.contains(split_y[-2])) &
            (df_events_vaep['matchId'] == int(x))]
    except:
        pass_df = df_events_vaep.loc[
            (df_events_vaep['playerName'].str.contains(split_y[-1]))
            & (df_events_vaep['matchId'] == int(x))]
    numpasses = len(pass_df)
    numaccpasses = len(
        pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)])
    accpasslocs = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Accurate" in a)]['positions'].tolist()
    inaccpasslocs = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Not accurate" in a)]['positions'].tolist()
    acc_vaep_values = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Accurate" in a)]['vaep_value'].tolist()
    inacc_vaep_values = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Not accurate" in a)]['vaep_value'].tolist()
    acc_off_values = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Accurate" in a)]['offensive_value'].tolist()
    inacc_off_values = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Not accurate" in a)]['offensive_value'].tolist()
    acc_def_values = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Accurate" in a)]['defensive_value'].tolist()
    inacc_def_values = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Not accurate" in a)]['defensive_value'].tolist()
    acc_vaep_values_norm = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Accurate" in a)]['vaep_value_norm'].tolist()
    inacc_vaep_values_norm = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Not accurate" in a)]['vaep_value_norm'].tolist()
    acc_off_values_norm = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Accurate" in a)]['offensive_value_norm'].tolist()
    inacc_off_values_norm = pass_df.loc[pass_df['tags'].apply(
        lambda a: "Not accurate" in a)]['offensive_value_norm'].tolist()
    return [
        numpasses, numaccpasses, accpasslocs, inaccpasslocs, acc_vaep_values,
        inacc_vaep_values, acc_off_values, inacc_off_values, acc_def_values,
        inacc_def_values, acc_vaep_values_norm, inacc_vaep_values_norm,
        acc_off_values_norm, inacc_off_values_norm
    ]

In [75]:
# getmetrics(2500081,"Bruno")

In [82]:
new_cols = ['RB_all',
            'R_CB_all',
            'L_CB_all',
            'LB_all',
            'RCB_all',
            'CB_all',
            'LCB_all',
            'RWB_all',
            'LWB_all']

**Collecting metrics for each defender location for various clusters**

In [83]:
#R_CB - Right center back for 4 defender formation
#RCB - Right center back for 3 or 5 defender formation
#L_CB - Left center back for 4 defender formation
#LCB - Left center back for 3 or 5 defender formation
df_clusters_updated = list()
for df in tqdm(df_clusters):
    df = df.reindex(columns = df.columns.tolist() + new_cols)
    if df.iloc[0]['backline'] == 4.0:     
        df['RB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RB), axis=1)
        df['R_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['R_CB']), axis=1)
        df['L_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['L_CB']), axis=1)
        df['LB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LB), axis=1)
        df_clusters_updated.append(df)
    
    elif df.iloc[0]['backline'] == 3.0:
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df_clusters_updated.append(df)
        
    elif df.iloc[0]['backline'] == 5.0:
        df['RWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RWB), axis=1)
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df['LWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LWB), axis=1)
        df_clusters_updated.append(df)

**Splitting the metrics into individual columns**

In [84]:
df_clusters_metrics = list()
for df in tqdm(df_clusters_updated):
    if df.iloc[0]['backline'] == 4.0:
        df[[
            'RB_pass', 'RB_accpass', 'RB_accpassloc', 'RB_inaccpassloc',
            'RB_accvaep', 'RB_inaccvaep', 'RB_accoff', 'RB_inaccoff',
            'RB_accdef', 'RB_inaccdef', 'RB_accvaep_norm', 'RB_inaccvaep_norm',
            'RB_accoff_norm', 'RB_inaccoff_norm'
        ]] = pd.DataFrame(df['RB_all'].to_list(), index=df.index)
        df[[
            'R_CB_pass', 'R_CB_accpass', 'R_CB_accpassloc',
            'R_CB_inaccpassloc', 'R_CB_accvaep', 'R_CB_inaccvaep',
            'R_CB_accoff', 'R_CB_inaccoff', 'R_CB_accdef', 'R_CB_inaccdef',
            'R_CB_accvaep_norm', 'R_CB_inaccvaep_norm', 'R_CB_accoff_norm',
            'R_CB_inaccoff_norm'
        ]] = pd.DataFrame(df['R_CB_all'].to_list(), index=df.index)
        df[[
            'L_CB_pass', 'L_CB_accpass', 'L_CB_accpassloc',
            'L_CB_inaccpassloc', 'L_CB_accvaep', 'L_CB_inaccvaep',
            'L_CB_accoff', 'L_CB_inaccoff', 'L_CB_accdef', 'L_CB_inaccdef',
            'L_CB_accvaep_norm', 'L_CB_inaccvaep_norm', 'L_CB_accoff_norm',
            'L_CB_inaccoff_norm'
        ]] = pd.DataFrame(df['L_CB_all'].to_list(), index=df.index)
        df[[
            'LB_pass', 'LB_accpass', 'LB_accpassloc', 'LB_inaccpassloc',
            'LB_accvaep', 'LB_inaccvaep', 'LB_accoff', 'LB_inaccoff',
            'LB_accdef', 'LB_inaccdef', 'LB_accvaep_norm', 'LB_inaccvaep_norm',
            'LB_accoff_norm', 'LB_inaccoff_norm'
        ]] = pd.DataFrame(df['LB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

    elif df.iloc[0]['backline'] == 3.0:
        df[[
            'RCB_pass', 'RCB_accpass', 'RCB_accpassloc', 'RCB_inaccpassloc',
            'RCB_accvaep', 'RCB_inaccvaep', 'RCB_accoff', 'RCB_inaccoff',
            'RCB_accdef', 'RCB_inaccdef', 'RCB_accvaep_norm',
            'RCB_inaccvaep_norm', 'RCB_accoff_norm', 'RCB_inaccoff_norm'
        ]] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[[
            'CB_pass', 'CB_accpass', 'CB_accpassloc', 'CB_inaccpassloc',
            'CB_accvaep', 'CB_inaccvaep', 'CB_accoff', 'CB_inaccoff',
            'CB_accdef', 'CB_inaccdef', 'CB_accvaep_norm', 'CB_inaccvaep_norm',
            'CB_accoff_norm', 'CB_inaccoff_norm'
        ]] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[[
            'LCB_pass', 'LCB_accpass', 'LCB_accpassloc', 'LCB_inaccpassloc',
            'LCB_accvaep', 'LCB_inaccvaep', 'LCB_accoff', 'LCB_inaccoff',
            'LCB_accdef', 'LCB_inaccdef', 'LCB_accvaep_norm', 'LCB_inaccvaep_norm',
            'LCB_accoff_norm', 'LCB_inaccoff_norm'
        ]] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

    elif df.iloc[0]['backline'] == 5.0:
        df[[
            'RCB_pass', 'RCB_accpass', 'RCB_accpassloc', 'RCB_inaccpassloc',
            'RCB_accvaep', 'RCB_inaccvaep', 'RCB_accoff', 'RCB_inaccoff',
            'RCB_accdef', 'RCB_inaccdef', 'RCB_accvaep_norm',
            'RCB_inaccvaep_norm', 'RCB_accoff_norm', 'RCB_inaccoff_norm'
        ]] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[[
            'CB_pass', 'CB_accpass', 'CB_accpassloc', 'CB_inaccpassloc',
            'CB_accvaep', 'CB_inaccvaep', 'CB_accoff', 'CB_inaccoff',
            'CB_accdef', 'CB_inaccdef', 'CB_accvaep_norm', 'CB_inaccvaep_norm',
            'CB_accoff_norm', 'CB_inaccoff_norm'
        ]] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[[
            'LCB_pass', 'LCB_accpass', 'LCB_accpassloc', 'LCB_inaccpassloc',
            'LCB_accvaep', 'LCB_inaccvaep', 'LCB_accoff', 'LCB_inaccoff',
            'LCB_accdef', 'LCB_inaccdef', 'LCB_accvaep_norm', 'LCB_inaccvaep_norm',
            'LCB_accoff_norm', 'LCB_inaccoff_norm'
        ]] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df[[
            'RWB_pass', 'RWB_accpass', 'RWB_accpassloc', 'RWB_inaccpassloc',
            'RWB_accvaep', 'RWB_inaccvaep', 'RWB_accoff', 'RWB_inaccoff',
            'RWB_accdef', 'RWB_inaccdef', 'RWB_accvaep_norm', 'RWB_inaccvaep_norm',
            'RWB_accoff_norm', 'RWB_inaccoff_norm'
        ]] = pd.DataFrame(df['RWB_all'].to_list(), index=df.index)
        df[[
            'LWB_pass', 'LWB_accpass', 'LWB_accpassloc', 'LWB_inaccpassloc',
            'LWB_accvaep', 'LWB_inaccvaep', 'LWB_accoff', 'LWB_inaccoff',
            'LWB_accdef', 'LWB_inaccdef', 'LWB_accvaep_norm', 'LWB_inaccvaep_norm',
            'LWB_accoff_norm', 'LWB_inaccoff_norm'
        ]] = pd.DataFrame(df['LWB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

In [85]:
df_clusters_metrics[0].head()

,wyId,team,team_defense,RB,R_CB,L_CB,LB,RCB,CB,LCB,RWB,LWB,backline,match,gameweek,teamsData,dateutc,venue,referees,score,footedness,temp,RB_pass,RB_accpass,RB_accpassloc,RB_inaccpassloc,RB_accvaep,RB_inaccvaep,RB_accoff,RB_inaccoff,RB_accdef,RB_inaccdef,RB_accvaep_norm,RB_inaccvaep_norm,RB_accoff_norm,RB_inaccoff_norm,R_CB_pass,R_CB_accpass,R_CB_accpassloc,R_CB_inaccpassloc,R_CB_accvaep,R_CB_inaccvaep,R_CB_accoff,R_CB_inaccoff,R_CB_accdef,R_CB_inaccdef,R_CB_accvaep_norm,R_CB_inaccvaep_norm,R_CB_accoff_norm,R_CB_inaccoff_norm,L_CB_pass,L_CB_accpass,L_CB_accpassloc,L_CB_inaccpassloc,L_CB_accvaep,L_CB_inaccvaep,L_CB_accoff,L_CB_inaccoff,L_CB_accdef,L_CB_inaccdef,L_CB_accvaep_norm,L_CB_inaccvaep_norm,L_CB_accoff_norm,L_CB_inaccoff_norm,LB_pass,LB_accpass,LB_accpassloc,LB_inaccpassloc,LB_accvaep,LB_inaccvaep,LB_accoff,LB_inaccoff,LB_accdef,LB_inaccdef,LB_accvaep_norm,LB_inaccvaep_norm,LB_accoff_norm,LB_inaccoff_norm
0,2500089,Burnley,"[MatthewLowton, KevinLong, JamesTarkowski, Ste...",MatthewLowton,KevinLong,JamesTarkowski,StephenWard,NaN,NaN,NaN,NaN,NaN,4,Burnley-Bournemouth,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",2018-05-13 14:00:00,Turf Moor,"[{'refereeId': 385705, 'role': 'referee'}, {'r...",1 - 2,right-right-right-left,2500089Burnley,38,25,"[[[33.28, 11.56], [20.8, 23.8]], [[12.48, 9.52...","[[[67.6, 6.8], [76.96, 0.0]], [[28.08, 8.84], ...","[0.0011006828863173723, 0.00801292434334755, 0...","[-0.007049093022942543, 0.0012342056725174189,...","[-0.0001301635056734085, -0.000179836759343743...","[-0.004554690793156624, 0.0018916875123977661,...","[0.0012308463919907808, 0.008192760869860649, ...","[-0.0024944019969552755, -0.000657481839880347...","[0.44277462363243103, 0.44665729999542236, 0.4...","[0.4381968379020691, 0.44284963607788086, 0.43...","[0.437611848115921, 0.4375837445259094, 0.4398...","[0.43511027097702026, 0.43875497579574585, 0.4...",45,39,"[[[20.8, 23.8], [7.28, 31.28]], [[10.4, 10.2],...","[[[37.44, 37.4], [63.44, 48.96]], [[45.76, 33....","[-0.0014702430926263332, 0.005621695891022682,...","[-0.004378382116556168, -0.005427076481282711,...","[-0.0010089944116771221, -0.000449455808848142...","[-0.0035393645521253347, -0.004665496293455362...","[-0.0004612486809492111, 0.006071151699870825,...","[-0.0008390173316001892, -0.000761580420657992...","[0.44133052229881287, 0.44531410932540894, 0.4...","[0.4396969974040985, 0.4391079545021057, 0.440...","[0.437114953994751, 0.43743130564689636, 0.438...","[0.43568432331085205, 0.43504759669303894, 0.4...",39,32,"[[[33.28, 63.92], [61.36, 67.32]], [[35.36, 57...","[[[36.4, 51.68], [69.68, 59.84]], [[7.28, 57.1...","[0.0026389099657535553, -1.3850978575646877e-0...","[-0.0026198537088930607, -0.005911990068852901...","[0.0030088734347373247, -0.0006259232759475708...","[-0.0017644562758505344, -0.000271227909252047...","[-0.0003699633525684476, 0.0006120722973719239...","[-0.0008553974330425262, -0.005640762392431498...","[0.4436386823654175, 0.4421485960483551, 0.443...","[0.44068479537963867, 0.4388355612754822, 0.44...","[0.43938660621643066, 0.43733152747154236, 0.4...","[0.4366878271102905, 0.43753206729888916, 0.43...",47,23,"[[[72.8, 63.92], [79.04, 65.96]], [[32.24, 57....","[[[43.68, 59.16], [74.88, 65.96]], [[102.96, 2...","[0.0004555538762360811, -0.00287072267383337, ...","[-0.0022187165450304747, -0.039802778512239456...","[0.0003273291513323784, -0.0023691633250564337...","[-0.0012894300743937492, -0.0397501215338707, ...","[0.00012822472490370274, -0.000501559232361614...","[-0.0009292864706367254, -5.265767686069012e-0...","[0.44241225719451904, 0.44054386019706726, 0.4...","[0.44091010093688965, 0.419798880815506, 0.439...","[0.43787050247192383, 0.4363459348678589, 0.43...","[0.43695640563964844, 0.41521114110946655, 0.4..."
3,2500097,Leicester,"[DannySimpson, WesMorgan, HarryMaguire, Christ...",DannySimpson,WesMorgan,HarryMaguire,ChristianFuchs,NaN,NaN,NaN,NaN,NaN,4,Spurs-Leicester,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 's

In [86]:
# Saving the clusters into individual pickle files
names = ['rrrl','rrll','rrl','rrr','rll','rrrll','rrlr','rrrr','rrrrl','rlr','rrrlr','rrlll','rlll']

for i,df in enumerate(df_clusters_metrics):
    df.to_pickle(f'../../../data/clusters/clusters_vaep/cluster_{names[i]}.pkl')

# Value sums per Team per Match 

In [58]:
vaep_values = pd.read_pickle('../../data/vaep/vaep_values.pkl')
match_def = pd.read_pickle("../../data/matches/match+def_lineup+footedness_ver2.pkl")

In [30]:
vaep_values = vaep_values.replace({'short_team_name':{
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }}
)

vaep_values['game_id'] = vaep_values['game_id'].astype(int)
vaep_values['temp'] = vaep_values['game_id'].astype(str) + vaep_values['short_team_name']
match_def['temp'] = match_def['wyId'].astype(str) + match_def['team']

In [31]:
vaep_values = vaep_values.merge(match_def[['temp', 'footedness']], left_on='temp', right_on='temp', how='left')
vaep_values.drop(columns = ['temp'], inplace=True)

In [32]:
vaep_values['name'] = vaep_values['first_name']+vaep_values['last_name']
vaep_values['name'] = vaep_values['name'].astype(str).apply(lambda x: unidecode(x))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace('-', ''))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace(' ', ''))
# vaep_values.loc[(vaep_values['last_name'].str.contains('PhilJa'))]

In [33]:
# Creating a key to identify each row using game_id, time_seconds and player name
vaep_values['key'] = vaep_values['game_id'].astype(str) + np.round(vaep_values['time_seconds'],6).astype(str) +vaep_values['name'].astype(str)

In [34]:
# vaep_values.loc[(vaep_values['name'].str.contains('AaronCre'))& (vaep_values['type_name']=='pass') &(vaep_values['game_id']==2499724)]


In [35]:
# Filtering the events that are labelled as pass and cross
vaep_values_pass = vaep_values#.loc[(vaep_values['type_name'] == 'pass') | (vaep_values['type_name'] =='cross') ]

In [37]:
# Merging player roles to SPADL events data
df_players = pd.read_pickle('../../data/players/players.pkl')
roles_temp = df_players['role'].values
roles = list()
for i in roles_temp:
    roles.append(i['code3'])
players_roles = list(zip(roles,df_players['wyId'],df_players['playerName']))
df_players_roles = pd.DataFrame(players_roles,columns = ['role','playerId','playerName1'])
vaep_values_pass_proles = vaep_values_pass.merge(df_players_roles, left_on = 'player_id', right_on = 'playerId')
vaep_values_pass_proles.drop(['playerName1'], axis = 1, inplace = True)
vaep_values_pass_def = vaep_values_pass_proles#.loc[vaep_values_pass_proles['role']=='DEF']

In [39]:
# Importing Wyscout events data
df_events_wyscout = pd.read_pickle('../../data/events/events_v2.pkl')

In [40]:
# Filtering out passes events which were performed by defenders in Wyscout events data
df_events_wyscout_pass = df_events_wyscout#.loc[(df_events_wyscout['eventName']=='Pass') & (df_events_wyscout['role']=='DEF')]

In [41]:
# Creating a key to identify each row using matchid, event_seconds and player name
df_events_wyscout_pass['key'] = df_events_wyscout_pass['matchId'].astype(str) + np.round(df_events_wyscout_pass['eventSec'],6).astype(str) + df_events_wyscout_pass['playerName'].astype(str)

In [43]:
# Appending VAEP values along with offensive and defensive values to Wyscout events data from VAEP data
# by joining the two dataframes
df_events_vaep = df_events_wyscout_pass.merge(vaep_values_pass_def[[
    'key', 'vaep_value', 'offensive_value', 'defensive_value',
    'vaep_value_norm', 'offensive_value_norm'
]],
                                              how='left',
                                              left_on='key',
                                              right_on='key')

In [44]:
df_events_vaep.head(20)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,wyId,foot,playerName,role,key,vaep_value,offensive_value,defensive_value,vaep_value_norm,offensive_value_norm
0,8,Simple pass,[Accurate],25413,"[[50.96, 34.68], [32.24, 14.96]]",2499719,Pass,1609,1H,2.758649,85,177959171,25413.0,right,AlexandreLacazette,FWD,24997192.758649AlexandreLacazette,0.000000,0.000000,-0.000000,0.442156,0.437685
1,1,Air duel,"[Lost, Not accurate]",25413,"[[73.84, 31.28], [52.0, 27.88]]",2499719,Duel,1609,1H,22.551816,10,177959181,25413.0,right,AlexandreLacazette,FWD,249971922.551816AlexandreLacazette,NaN,NaN,NaN,NaN,NaN
2,10,Shot,"[Goal, Right foot, Opportunity, Position: Goal...",25413,"[[91.52, 40.12], [0.0, 68.0]]",2499719,Shot,1609,1H,94.595788,100,177959212,25413.0,right,AlexandreLacazette,FWD,249971994.595788AlexandreLacazette,0.931202,0.930000,0.001202,0.965219,0.963497
3,8,Head pass,[Not accurate],25413,"[[73.84, 20.4], [73.84, 25.16]]",2499719,Pass,1609,1H,397.881307,82,177959276,25413.0,right,AlexandreLacazette,FWD,2499719397.881307AlexandreLacazette,-0.009786,-0.007210,-0.002576,0.436659,0.433609
4,1,Ground defending duel,"[Take on left, Lost, Not accurate]",25413,"[[63.44, 24.48], [53.04, 8.84]]",2499719,Duel,1609,1H,494.461238,12,177959303,25413.0,right,AlexandreLacazette,FWD,2499719494.461238AlexandreLacazette,NaN,NaN,NaN,NaN,NaN
5,8,Simple pass,[Not accurate],25413,"[[31.2, 42.16], [29.12, 68.0]]",2499719,Pass,1609,1H,752.105957,85,177959390,25413.0,right,AlexandreLacazette,FWD,2499719752.105957AlexandreLacazette,-0.005411,-0.004813,-0.000598,0.439117,0.434964
6,1,Ground defending duel,"[Won, Accurate]",25413,"[[92.56, 17.68], [79.04, 14.96]]",2499719,Duel,1609,1H,808.662553,12,177959421,25413.0,right,AlexandreLacazette,FWD,2499719808.662553AlexandreLacazette,NaN,NaN,NaN,NaN,NaN
7,1,Ground attacking duel,"[Counter attack, Take on left, Won, Accurate]",25413,"[[64.48, 47.6], [64.48, 48.96]]",2499719,Duel,1609,1H,1118.937573,11,177959537,25413.0,right,AlexandreLacazette,FWD,24997191118.937573AlexandreLacazette,0.015767,0.015965,-0.000198,0.451013,0.446712
8,8,Simple pass,[Accurate],25413,"[[74.88, 45.56], [63.44, 52.36]]",2499719,Pass,1609,1H,1253.590895,85,177959580,25413.0,right,AlexandreLacazette,FWD,24997191253.590895AlexandreLacazette,-0.006865,-0.006741,-0.000124,0.438301,0.433874
9,1,Ground defending duel,"[Neutral, Accurate]",25413,"[[59.28, 12.92], [60.32, 13.6]]",2499719,Duel,1609,1H,1342.374693,12,177959628,25413.0,right,AlexandreLacazette,FWD,24997191342.374693AlexandreLacazette,NaN,NaN,NaN,NaN,NaN


**Creating DF for Value sums per team per match**

In [45]:
value_sums = pd.DataFrame(df_events_vaep.groupby(['matchId', 'teamId']))

In [48]:
def value_sum_regionwise(df):
    off_sum = [0,0,0,0]
    vaep_sum = [0,0,0,0]
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    for i in range(len(df)):
        try:
            if (df['positions'][i][1][0] >= 52) and (df['positions'][i][1][1] <= 17):
                off_sum[0]+= df['offensive_value_norm'].loc[i]
                vaep_sum[0]+= df['vaep_value_norm'].loc[i]
            elif (df['positions'][i][1][0] >= 52) and (df['positions'][i][1][1] > 17) and (df['positions'][i][1][1] <= 34):
                off_sum[1]+= df['offensive_value_norm'].loc[i]
                vaep_sum[1]+= df['vaep_value_norm'].loc[i]
            elif (df['positions'][i][1][0] >= 52) and (df['positions'][i][1][1] > 34) and (df['positions'][i][1][1] <= 51):
                off_sum[2]+= df['offensive_value_norm'].loc[i]
                vaep_sum[2]+= df['vaep_value_norm'].loc[i]
            elif (df['positions'][i][1][0] >= 52) and (df['positions'][i][1][1] > 51):
                off_sum[3]+= df['offensive_value_norm'].loc[i]
                vaep_sum[3]+= df['vaep_value_norm'].loc[i]
            else:
                continue
        except:
            print(i)
            
    return off_sum, vaep_sum


In [49]:
value_sums['offsum_regionwise'] = value_sums[1].apply(lambda x: value_sum_regionwise(x)[0])
value_sums['vaepsum_regionwise'] = value_sums[1].apply(lambda x: value_sum_regionwise(x)[1])

148
189
350
379
214
348
244
540
393
573
5
168
623
557
718
123
323
111
501
150
295
67
294
498
386
478
542
282
306
325
392
364
431
604
552
329
402
291
528
184
211
409
119
384
234
430
139
67
293
227
72
189
164
70
137
755
85
282
103
135
471
610
303
241
407
617
216
317
81
143
9
376
167
213
348
202
129
401
378
292
261
138
416
223
430
409
866
47
287
289
136
17
51
260
22
523
184
18
299
429
275
105
205
245
486
461
335
248
429
176
419
69
162
46
394
106
321
356
148
189
350
379
214
348
244
540
393
573
5
168
623
557
718
123
323
111
501
150
295
67
294
498
386
478
542
282
306
325
392
364
431
604
552
329
402
291
528
184
211
409
119
384
234
430
139
67
293
227
72
189
164
70
137
755
85
282
103
135
471
610
303
241
407
617
216
317
81
143
9
376
167
213
348
202
129
401
378
292
261
138
416
223
430
409
866
47
287
289
136
17
51
260
22
523
184
18
299
429
275
105
205
245
486
461
335
248
429
176
419
69
162
46
394
106
321
356


In [50]:
value_sums.head()

,0,1,offsum_regionwise,vaepsum_regionwise
0,"(2499719, 1609)",eventId subEventName \ 0 ...,"[43.77355965971947, 44.83893629908562, 60.4206...","[44.21892383694649, 45.27823010087013, 60.9986..."
1,"(2499719, 1631)",eventId subEventName \ 0 ...,"[36.23892968893051, 15.255093157291412, 22.550...","[36.531051099300385, 15.395004719495773, 22.73..."
2,"(2499720, 1625)",eventId subEventName \ 0 ...,"[83.86989837884903, 65.73087322711945, 62.0668...","[84.63515478372574, 66.36549571156502, 62.6686..."
3,"(2499720, 1651)",eventId subEventName \ 0 ...,"[25.373297691345215, 10.58007875084877, 12.267...","[25.606199830770493, 10.665535300970078, 12.38..."
4,"(2499721, 1610)",eventId subEventName \ 0 ...,"[41.628524631261826, 48.530882596969604, 42.90...","[42.07614079117775, 48.95190331339836, 43.2980..."


In [52]:
teams = pd.read_json('../../data/teams/teams.json')

In [53]:
value_sums['team_name'] = value_sums[0].apply(lambda x: teams[teams['wyId']==x[1]]['name'].values[0])

In [54]:
value_sums['match_id'] = value_sums[0].apply(lambda x: x[0])

In [55]:
value_sums = value_sums.replace({'team_name':{
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }}
)

In [56]:
value_sums.drop(columns=[0,1], inplace=True)
value_sums.head()

,offsum_regionwise,vaepsum_regionwise,team_name,match_id
0,"[43.77355965971947, 44.83893629908562, 60.4206...","[44.21892383694649, 45.27823010087013, 60.9986...",Arsenal,2499719
1,"[36.23892968893051, 15.255093157291412, 22.550...","[36.531051099300385, 15.395004719495773, 22.73...",Leicester,2499719
2,"[83.86989837884903, 65.73087322711945, 62.0668...","[84.63515478372574, 66.36549571156502, 62.6686...",Man City,2499720
3,"[25.373297691345215, 10.58007875084877, 12.267...","[25.606199830770493, 10.665535300970078, 12.38...",Brighton,2499720
4,"[41.628524631261826, 48.530882596969604, 42.90...","[42.07614079117775, 48.95190331339836, 43.2980...",Chelsea,2499721


In [57]:
value_sums.to_pickle('../../data/vaep/value_sums.pkl')